In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import StringType
from pyspark import SQLContext
import numpy as np

In [2]:
conf = SparkConf().setMaster("local").setAppName("Mi programa")
sc = SparkContext(conf = conf)
sqlContext = SQLContext(sc)


dfspark = sqlContext.read.format("csv").option("header", "true").option("inferSchema","true").load("C:/Users/lucho/Desktop/Programacion/Python/Linkedin Data Science/base_datos_2008.csv")
dfspark = dfspark.sample(fraction = 0.001, withReplacement = False)
dfspark = dfspark.withColumn("ArrDelay",dfspark["ArrDelay"].cast("integer"))
dfspark = dfspark.withColumn("DepDelay",dfspark["DepDelay"].cast("integer"))

In [5]:
df2 = dfspark.na.drop(subset=["ArrDelay","DepDelay","Distance"])
df2 = df2.filter("ArrDelay is not null")
df2 = df2.dropDuplicates()

In [7]:
df2.select("ArrDelay").filter("ArrDelay > 60").take(5)

[Row(ArrDelay=100),
 Row(ArrDelay=71),
 Row(ArrDelay=96),
 Row(ArrDelay=73),
 Row(ArrDelay=86)]

In [8]:
df2.filter("ArrDelay > 60").take(5)

[Row(Year=2008, Month=2, DayofMonth=26, DayOfWeek=2, DepTime='1202', CRSDepTime=1015, ArrTime='1305', CRSArrTime=1125, UniqueCarrier='MQ', FlightNum=3985, TailNum='N690AE', ActualElapsedTime='63', CRSElapsedTime='70', AirTime='44', ArrDelay=100, DepDelay=107, Origin='ORD', Dest='RST', Distance=268, TaxiIn='4', TaxiOut='15', Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay='0', WeatherDelay='23', NASDelay='0', SecurityDelay='0', LateAircraftDelay='77'),
 Row(Year=2008, Month=5, DayofMonth=11, DayOfWeek=7, DepTime='718', CRSDepTime=600, ArrTime='1037', CRSArrTime=926, UniqueCarrier='9E', FlightNum=3756, TailNum='80209E', ActualElapsedTime='139', CRSElapsedTime='146', AirTime='101', ArrDelay=71, DepDelay=78, Origin='TUL', Dest='DTW', Distance=790, TaxiIn='21', TaxiOut='17', Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay='71', WeatherDelay='0', NASDelay='0', SecurityDelay='0', LateAircraftDelay='0'),
 Row(Year=2008, Month=6, DayofMonth=13, DayOfWeek=5, DepTime='14

In [10]:
media = np.mean(df2.select("ArrDelay").collect())
df2.select("ArrDelay").rdd.map(lambda x: (x-media)**2).take(10)

[array([0.65214766]),
 array([368.34988362]),
 array([368.34988362]),
 array([23.11260047]),
 array([14.49748727]),
 array([8428.62744905]),
 array([174.04056284]),
 array([14.49748727]),
 array([4.80680806]),
 array([51.73124205])]

In [12]:
df2.groupBy("DayOfWeek").count().show()

+---------+-----+
|DayOfWeek|count|
+---------+-----+
|        1|  996|
|        6|  764|
|        3| 1051|
|        5| 1017|
|        4| 1016|
|        7|  921|
|        2| 1037|
+---------+-----+



In [14]:
df2.groupBy("DayOfWeek").mean("ArrDelay").show()

+---------+------------------+
|DayOfWeek|     avg(ArrDelay)|
+---------+------------------+
|        1| 7.833333333333333|
|        6|3.9018324607329844|
|        3| 6.902949571836347|
|        5|10.731563421828909|
|        4| 8.356299212598426|
|        7|10.881650380021716|
|        2|7.9662487945998075|
+---------+------------------+



In [17]:
df2.select("Origin").rdd.distinct().take(5)

[Row(Origin='PIT'),
 Row(Origin='FSM'),
 Row(Origin='SPI'),
 Row(Origin='SMF'),
 Row(Origin='DLH')]

In [ ]:
df2.select("Origin").rdd.distinct().count()